## TUTORIAL 06 - Unsteady Thermal block problem
**_Keywords: POD-Galerkin method, scalar problem_**

### 1. Introduction
In this Tutorial, we consider unsteady heat conduction in a two-dimensional domain $\Omega$.

<img src="data/thermal_block.png" />

We define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk centered at the origin of radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$. 

The conductivity $\kappa$ is assumed to be constant on $\Omega_1$ and $\Omega_2$, i.e.
$$
\kappa|_{\Omega_1}=\kappa_0 \quad \textrm{and} \quad \kappa|_{\Omega_2}=1.
$$

For this problem, we consider $P=2$ parameters:
1. the first one is related to the conductivity in $\Omega_1$, i.e. $\mu_0\equiv\kappa_0$ (_note that parameters numbering is zero-based_);
2. the second parameter $\mu_1$ takes into account the constant heat flux over $\Gamma_{base}$.

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10]\times[-1,1].
$$

In this problem we model the heat transfer process due to the heat flux over the bottom boundary $\Gamma_{base}$ and the following conditions on the remaining boundaries:
* the left and right boundaries $\Gamma_{side}$ are insulated,
* the top boundary $\Gamma_{top}$ is kept at a reference temperature (say, zero),

with the aim of measuring the average temperature on $\Gamma_{base}$.

In order to obtain a faster approximation of the problem we pursue a model reduction by means of a POD-Galerkin reduced order method.

### 2. Parametrized formulation

Let $u(t;\boldsymbol{\mu})$ be the temperature in the domain $\Omega\times[0,t_f]$.

The strong formulation of the parametrized problem is given by:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, for $t\in[0,t_f]$, find $u(t;\boldsymbol{\mu})$ such that</center>

$$
\begin{cases}
	\partial_tu(t;\boldsymbol{\mu})- \text{div} (\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})) = 0 & \text{in } \Omega\times[0,t_f],\\
    u(t=0;\boldsymbol{\mu}) = 0 & \text{in } \Omega, \\ 
	u(t;\boldsymbol{\mu}) = 0 & \text{on } \Gamma_{top}\times[0,t_f],\\
	\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})\cdot \mathbf{n} = 0 & \text{on } \Gamma_{side}\times[0,t_f],\\
	\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})\cdot \mathbf{n} = \mu_1 & \text{on } \Gamma_{base}\times[0,t_f].
\end{cases}
$$
<br>

where 
* $\mathbf{n}$ denotes the outer normal to the boundaries $\Gamma_{side}$ and $\Gamma_{base}$,
* the conductivity $\kappa(\mu_0)$ is defined as follows:
$$
\kappa(\mu_0) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	1 & \text{in } \Omega_2,\\
\end{cases}
$$

The corresponding weak formulation reads:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, for $t\in[0,t_f]$, find $u(t;\boldsymbol{\mu})\in\mathbb{V}$ such that</center>

$$m\left(\partial_tu(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right) + a\left(u(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V},\quad \forall t\in[0,t_f]$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H^1(\Omega) : v|_{\Gamma_{top}}=0\}
$$
* the parametrized bilinear form $m(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$m(u, v;\boldsymbol{\mu})=\int_{\Omega} \partial_tu(t)v \ d\boldsymbol{x},$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \kappa(\mu_0)\nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \mu_1\int_{\Gamma_{base}}v \ ds,$$

The (compliant) output of interest $s(t;\boldsymbol{\mu})$ is given by
$$s(t;\boldsymbol{\mu}) = \mu_1\int_{\Gamma_{base}} u(t;\boldsymbol{\mu})$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
import sys
sys.path.append("../../MLniCS/")

from mlnics import NN, Losses, Normalization, RONNData
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

## 3. Affine decomposition

For this problem the affine decomposition is straightforward:
$$m(u,v;\boldsymbol{\mu})=\underbrace{1}_{\Theta^{m}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}uv \ d\boldsymbol{x}}_{m_0(u,v)},$$
$$a(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega_1}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)} \ + \  \underbrace{1}_{\Theta^{a}_1(\boldsymbol{\mu})}\underbrace{\int_{\Omega_2}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_1(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{\mu_1}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Gamma_{base}}v \ ds}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class UnsteadyThermalBlock(ParabolicCoerciveProblem):
```
by specifying the coefficients $\Theta^{m}_*(\boldsymbol{\mu})$, $\Theta^{a}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):     
```
and the bilinear forms $m_*(u, v)$, $a_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
class UnsteadyThermalBlock(ParabolicCoerciveProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        ParabolicCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)

    # Return custom problem name
    def name(self):
        return "UnsteadyThermalBlock1POD"

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu = self.mu
        if term == "m":
            theta_m0 = 1.
            return (theta_m0, )
        elif term == "a":
            theta_a0 = mu[0]
            theta_a1 = 1.
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = mu[1]
            return (theta_f0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "m":
            u = self.u
            m0 = u * v * dx
            return (m0, )
        elif term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx(1)
            a1 = inner(grad(u), grad(v)) * dx(2)
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            f0 = v * ds(1)
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        elif term == "projection_inner_product":
            u = self.u
            x0 = u * v * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/thermal_block.xml")
subdomains = MeshFunction("size_t", mesh, "data/thermal_block_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/thermal_block_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1, two components)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the UnsteadyThermalBlock class

In [5]:
problem = UnsteadyThermalBlock(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)
problem.set_time_step_size(0.05)
problem.set_final_time(3)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20, nested_POD=4)
reduction_method.set_tolerance(1e-8, nested_POD=1e-4)

### 4.5. Perform the offline phase

In [19]:
reduction_method.initialize_training_set(5)
reduced_problem = reduction_method.offline()

In [20]:
net = NN.RONN(problem, reduction_method)

In [21]:
input_normalization = Normalization.StandardNormalization()
output_normalization = Normalization.IdentityNormalization()

In [22]:
pdnn_loss = Losses.PDNN_Loss(net, output_normalization)
pinn_loss = Losses.PINN_Loss(net)
prnn_loss = Losses.PRNN_Loss(net, output_normalization, omega=1.)

In [23]:
data = RONNData.RONNDataLoader(net)
_ = data.train_validation_split(0.2)

In [25]:
_ = NN.normalize_and_train(net, data, pdnn_loss, input_normalization, lr=0.0001, epochs=100000, use_validation=True)

0 0.014700570772695635 	Loss(validation) = 0.006597383032224384
100 0.002356241016150401 	Loss(validation) = 0.0016363522061047361
200 0.0013027929609287076 	Loss(validation) = 0.002048728257068842
300 0.000851473296683977 	Loss(validation) = 0.002125604975800791
400 0.0006528339057109529 	Loss(validation) = 0.001978458402428058
500 0.0005418565592725197 	Loss(validation) = 0.0018117422439414392
600 0.00047706305509712354 	Loss(validation) = 0.0016978857945724917
700 0.00043942294910128343 	Loss(validation) = 0.0016328892253928723
800 0.00041572588495844423 	Loss(validation) = 0.0016026911510468578
900 0.0003981899708253093 	Loss(validation) = 0.0015961763051387235
1000 0.0003832053064622132 	Loss(validation) = 0.0016060895295136961
1100 0.00036937682098203765 	Loss(validation) = 0.0016265509198436538
1200 0.00035604383403911165 	Loss(validation) = 0.0016539129846682844
1300 0.0003426670526813266 	Loss(validation) = 0.00168542219720353
1400 0.0003286496339107026 	Loss(validation) = 0.0

12000 2.694554924605093e-06 	Loss(validation) = 0.005609027288433818
12100 2.5245896623573627e-06 	Loss(validation) = 0.005608069910816141
12200 2.407929442383707e-06 	Loss(validation) = 0.005606324496064116
12300 2.3544176331587384e-06 	Loss(validation) = 0.005605348027886969
12400 2.288146618089722e-06 	Loss(validation) = 0.0056070244173325266
12500 2.344963168546253e-06 	Loss(validation) = 0.005596439606586671
12600 2.198206090963251e-06 	Loss(validation) = 0.0056065209507226215
12700 2.7158153066429583e-06 	Loss(validation) = 0.005602262758432706
12800 2.0713162964020783e-06 	Loss(validation) = 0.005608245087268288
12900 2.020617009983726e-06 	Loss(validation) = 0.005609489365310974
13000 1.98584710847915e-06 	Loss(validation) = 0.005609333331999727
13100 2.6077295087681877e-06 	Loss(validation) = 0.005616872983174666
13200 1.9933758959139563e-06 	Loss(validation) = 0.005609948558438252
13300 1.9077477378351993e-06 	Loss(validation) = 0.0056138979025199005
13400 1.7960447026802977e

23900 3.7557828944318073e-07 	Loss(validation) = 0.005836249372345913
24000 3.775656737282938e-07 	Loss(validation) = 0.0058384395394203495
24100 4.0389678010808006e-07 	Loss(validation) = 0.0058372843034608865
24200 1.2520768638085877e-06 	Loss(validation) = 0.005850137561304534
24300 3.652701946275406e-07 	Loss(validation) = 0.005840544485493545
24400 3.7237693561781943e-07 	Loss(validation) = 0.005838915794685828
24500 3.5474729228370764e-07 	Loss(validation) = 0.005841807348269626
24600 3.5154007716577456e-07 	Loss(validation) = 0.005842754026079298
24700 3.4992259158217377e-07 	Loss(validation) = 0.00584293556031802
24800 3.4725777991524704e-07 	Loss(validation) = 0.005843866274078455
24900 3.4184368195915635e-07 	Loss(validation) = 0.005844582974975973
25000 6.66975627164412e-07 	Loss(validation) = 0.0058500794326775095
25100 7.608828318224301e-07 	Loss(validation) = 0.0058556511468307055
25200 3.385863678763654e-07 	Loss(validation) = 0.005847348135528712
25300 4.172226017156701

35700 3.9634657968010716e-07 	Loss(validation) = 0.005904729107248769
35800 1.9236598954353964e-07 	Loss(validation) = 0.005909019201458459
35900 1.5551451613366873e-07 	Loss(validation) = 0.005907843724729854
36000 1.5669266388319848e-07 	Loss(validation) = 0.005907238197413568
36100 1.6282079821721065e-07 	Loss(validation) = 0.005911052833242513
36200 1.6502778554444282e-07 	Loss(validation) = 0.005908131375284247
36300 2.0599247703040697e-07 	Loss(validation) = 0.005908567797910908
36400 4.638700706083611e-07 	Loss(validation) = 0.005908959189749213
36500 8.151232382889903e-07 	Loss(validation) = 0.005905779462921243
36600 3.105540405902589e-07 	Loss(validation) = 0.00590513662354981
36700 1.5578471330944422e-07 	Loss(validation) = 0.005908065519563523
36800 2.221243589164246e-07 	Loss(validation) = 0.0059129029519374935
36900 1.5030052572305164e-07 	Loss(validation) = 0.0059108289436646776
37000 2.8251621895331765e-07 	Loss(validation) = 0.005907586661028566
37100 1.572842184972748

KeyboardInterrupt: 

### 4.6. Perform an online solve

In [26]:
online_mu = (8.0, -1.0)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem, every=5, interval=500)

In [30]:
net.eval()

with torch.no_grad():
    online_mu_nn = torch.tensor(online_mu)
    # NOTE: Change output_normalization accordingly for the different losses
    reduced_solution_nn = net.solve(online_mu_nn, input_normalization, output_normalization=None)

net.train()
plot(reduced_solution_nn, reduced_problem=reduced_problem, every=5, interval=500)

In [32]:
test = NN.get_test(net)
NN.error_analysis_fixed_net(net, test, input_normalization)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 0 is different from 6)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(10)
reduction_method.error_analysis()

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.initialize_testing_set(10)
reduction_method.speedup_analysis()